In [18]:
from pandas.io.parsers import read_csv
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_dangerously_set_inner_html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import pandas as pd
import numpy as np
import json
import requests
import dash_table
import random as r

#import data preprocessing function
from process_data import get_chart_ready_df, up_to_date_check

#Bootstrap frontend framework
import dash_bootstrap_components as dbc

#css sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css', dbc.themes.BOOTSTRAP]
# custom_css = requests.get('./stylesheets/style.css').

#for development
from jupyter_dash import JupyterDash

In [19]:
#Complete all data preprocessing if data is out-of-date
if up_to_date_check() : get_chart_ready_df()
    
df_map = pd.read_csv('../api_data/chart_ready.csv', parse_dates = ['Date'], index_col = 0)

#Last Updated variable
last_updated = df_map['Date'].max().date()
#Create Labels for Date Slider
#Get Unique Dates
unique_dates = pd.DataFrame(sorted(df_map['Date'].unique())).reset_index()
unique_dates.rename(columns = {'index':'id', 0 : 'Date'}, inplace = True)

#Merge map data with unique dates to assign 'id' value (for rendering choropleth map)
df_map = pd.merge(df_map, unique_dates, how = 'left', on = 'Date')

#Set Slider Range
slider_min = df_map['id'].min()
slider_max = df_map['id'].max()

day_filter = df_map['Day'] == 1
slider_tick_cols  = df_map.loc[day_filter,['id', 'Month and Year']].drop_duplicates()
slider_tick_labels = dict(slider_tick_cols.values)

#helper method for rendering top 15 most affected countries for a given date
def top_15(date = '2020-01-22', metric = 'Deaths'):
    return df_map.loc[df_map['Date'] == date, ['Country', metric]]\
    .sort_values(metric, ascending = False).head(15)
start_top_15 = top_15()

In [20]:
#Initialize Dash App
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

In [21]:
#Default - for troubleshooting when there are layout errors
app.layout = html.Div([

    dbc.Row(
        dbc.Col(
            html.Div( 
                style = {'background-color':'darkslategrey'},
                children = 
                    html.H3(['Data for this graph is provided by the Coronavirus Tracker API: ', 
                            'https://github.com/ExpDev07/coronavirus-tracker-api',
                             html.Br(), 
                             'Last Updated = {}'.format(last_updated.date)
                            ], 
                    style = {
                        'color' : '#ede7c7',
                        'font-size' : '14px', 
                        'font-weight' : 'bold'}            
                            )
                )
        )
    ),
    
    dbc.Container([
        
    dbc.Row(
        children = [
            dbc.Col(dcc.Graph(id = 'map'), 
                    width = 12),
            dbc.Col(dash_dangerously_set_inner_html.DangerouslySetInnerHTML('''
                        <p>Please use the slider and different metrics to explore
                        how the pandemic has evolved over time. 
                        The table below shows the worst-affected countries on a given day 
                        according and for the selected metric. Credit goes to Johns Hopkins 
                        University for collecting this data, and for the folks at 
                        <a href = 'https://github.com/ExpDev07/coronavirus-tracker-api'>
                        Open Coronavirus API </a> for creating the API.</p> 
                        ''' 
                    ),
                    style = {
                        'font-size':'14px'
                    }

                   ,
                    width = 12),
            dbc.Col(
                dcc.Slider(
                    id ='date-slider',
                    min = slider_min,
                    max = slider_max,
                    step = 1,
                    value = 0,
                    marks = slider_tick_labels),
                width = 12, 
                style = {
                    'padding' : '20px 0'
                })]
    )]),
    
     dbc.Container([
         dbc.Row(
        children = [
            dbc.Col(
                dcc.RadioItems(
                    id = 'metric_toggle',
                    options = [
                        {'label':'Crude Mortality', 'value': 'Deaths'},
                        {'label':'Mortality Per 1M', 'value': 'Deaths per 1M'},
                        {'label':'Crude Cases', 'value': 'Cases'},
                        {'label':'Case Rate per 1M', 'value': 'Cases per 1M'},    
                    ],
                    inputStyle = {'margin': '5px 25px 5px 0'},
                    labelStyle={'display': 'inline-block', 
                               'font-size' : 16},
                    value='Deaths'),
                width = 12,
                ), 
            
            dbc.Col(
                dash_table.DataTable(
                    id='top_15',
                    columns = [ {'name' : i , 'id' : i} for i in start_top_15.columns],
                    data=start_top_15.to_dict('records'),
                    style_as_list_view = True,
                    style_table={'height': '300px', 'overflowY': 'auto'},
                    fixed_rows={'headers': True},

                    style_data = {
                        'padding' : '5px',
                        'font_size': '12px',
                        'text_align': 'left'
                    },
                    style_header = {
                        'backgroundColor': 'white',
                        'padding' : '5px',
                        'font_size': '16px',
                        'text_align': 'left', 
                        'font_weight' : 'bold'
                    }
                ),
                style = {'height': '400px'}, 
            md = 12,
            lg = 4),
            
            dbc.Col(dcc.Graph(id = 'line-chart'), 
                    width = 12,
                    md = 12, 
                    lg = 8
                   )
        ])
         
     ])
])


AttributeError: 'datetime.date' object has no attribute 'date'

In [ ]:
#Callback function and Plotly components

@app.callback(
    Output('map', 'figure'),
    [Input(component_id = 'date-slider', component_property = 'value'),
    Input(component_id = 'metric_toggle', component_property = 'value')]
)
def update_map(new_date, new_metric):
    
    #helper function for color scale
    def max_range(x,y):
        return (x if x > y else y)
    
    filtered_df = df_map[df_map['id']  == new_date]
    color_scale_min = filtered_df[new_metric].min()
    color_scale_max = max_range(filtered_df[new_metric].max(), 200)
    
    fig = px.choropleth(filtered_df,
                locations = "ISO-3",               
                color = new_metric,
                hover_name = "Country",  
                color_continuous_scale = 'Amp',
                range_color = (color_scale_min, color_scale_max),
                title = 'Cumulative COVID-19 {} by Country'.format(new_metric), 
                template = 'plotly_white'
    )
    
    fig.update_geos(projection_type="natural earth")
    fig.update_layout(height=300, margin={"r":10,"t":30,"l":10,"b":30})
    fig.update_layout(transition_duration=500)
    fig.update_layout(title_x=0.3)

    return fig


@app.callback(
Output('line-chart', 'figure'),
    [Input(component_id = 'date-slider', component_property = 'value'),
    Input(component_id = 'metric_toggle', component_property = 'value')]
)
def update_line_chart(new_date, new_metric):
    
    line_chart_titles = { 
        'Cases' : "COVID-19 Cumulative Case Count per Country", 
        'Cases per 1M' : 'COVID-19 Cumulative Population - Adjusted Cases (Per 1M)',
        'Deaths': "COVID-19 Cumulative Death Count per Country",
        'Deaths per 1M' : 'COVID-19 Cumulative Population - Adjusted Deaths (Per 1M)'
    }
    
    top_15 = top_15(new_date, new_metric)
    
    fig = px.line(top_15,
        x = 'Date', 
        y = new_metric, 
        title = line_chart_titles[new_metric], 
        color = 'Country',
        text = new_metric, 
        template='gridon', 
        log_y = False,
        labels = dict(
            x = 'Date', 
            y = str(new_metric).title(),
            country = 'Country')
        )
    fig.update_traces(mode='lines')

    return fig



@app.callback(
    Output('top_15', 'data'), 
    [Input(component_id = 'date-slider', component_property = 'value'),
    Input(component_id = 'metric_toggle', component_property = 'value')]
)
def update_top15_data(new_date, new_metric):
    return df_map.loc[df_map['id'] == new_date, ['Country', new_metric]] \
    .sort_values(new_metric, ascending = False).head(15).to_dict('records')


@app.callback(
    Output('top_15', 'columns'), 
    [Input(component_id = 'date-slider', component_property = 'value'),
    Input(component_id = 'metric_toggle', component_property = 'value')]
)
def update_top15_headers(new_date, new_metric):
    return [{'name' : i , 'id' : i} for i in ['Country', new_metric]]
    

In [22]:
app.run_server(mode = 'external', port = 8099)
#add date to tooltip :)

OSError: Address 'http://127.0.0.1:8099' already in use.
    Try passing a different port to run_server.

In [131]:
#Check themes during development
#import plotly
# plotly.io.templates

['ggplot2',
 'seaborn',
 'simple_white',
 'plotly',
 'plotly_white',
 'plotly_dark',
 'presentation',
 'xgridoff',
 'ygridoff',
 'gridon',
 'none']

In [12]:
top_15('2020-05-05', 'Deaths').loc[:, 'Country']

54050     United States
25412             Italy
53717    United Kingdom
47723             Spain
18086            France
5432            Belgium
7763             Brazil
19418           Germany
24080              Iran
36068       Netherlands
11093             China
9761             Canada
52385            Turkey
49055            Sweden
33404            Mexico
Name: Country, dtype: object